array([0.75659366, 0.29472705, 0.37125099, ..., 0.96596137, 0.45535616,
       0.0046756 ])

In [2]:
import tensorflow as tf
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)

In [4]:
encoder = info.features['text'].encoder
print ('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [5]:
a = []
for i in range(14,32):
    try:
        for j in range(100):
            with open("../TrainingData/TeslaTrainingData_2019-10-"+str(i)+"/Tesla"+str(j)+".txt", mode='rb') as file:
                try:
                    a.append(str(file.read()))
                except:
                    continue
    except:
        continue
for i in range(1,14):
    try:
        for j in range(100):
            with open("../TrainingData/TeslaTrainingData_2019-11-"+str(i)+"/Tesla"+str(j)+".txt", mode='rb') as file:
                try:
                    a.append(str(file.read()))
                except:
                    continue
    except:
        continue

In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers
x_train = np.array(a, dtype=np.str)
x_train = x_train.astype(str)
print(x_train.shape)
y_train = np.zeros(2300)+0.5
print(y_train.shape)
import tensorflow as tf
embed_size = 300     # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
model1 = tf.keras.Sequential()
model1.add(Embedding(max_features, embed_size, input_length=maxlen))
model1.add(Bidirectional(LSTM(128, return_sequences=True)))
model1.add(GlobalMaxPool1D())
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1), metrics=['accuracy'])
model1.summary()
x_trainR = np.array(x_train)[:1900]
y_trainR = np.array(y_train)[:1900]
x_test = np.array(x_train)[1900:]
y_test = np.array(y_train)[1900:]
x_train = x_trainR
y_train = y_trainR
print("J")
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

(2300,)
(2300,)
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_7 (Bidirection (None, 100, 256)          439296    
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 256)               0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_22 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 32

In [ ]:
model1.fit(x_train, y_train, epochs=5, batch_size=10, validation_data=(x_test,y_test))

Train on 1900 samples, validate on 400 samples
Epoch 1/5
1900/1900 [==============================] - 50s 26ms/sample - loss: 0.6931 - accuracy: 0.0000e+00 - val_loss: 0.6931 - val_accuracy: 0.0000e+00
Epoch 2/5
1310/1900 [===================>..........] - ETA: 14s - loss: 0.7218 - accuracy: 0.0000e+00

In [ ]:
for i in range(9):
    encoder.encode(x_train[i])

In [9]:
def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec

In [12]:
def sample_predict(sentence, pad):
    encoded_sample_pred_text = encoder.encode(sample_pred_text)

    if pad:
        encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    predictions = model1.predict(tf.expand_dims(encoded_sample_pred_text, 0))

    return (predictions)

In [13]:
# predict on a sample text without padding.

sample_pred_text = ('Tesla is doing really well now. It is going to be one of the greatest companies ever in the near future')
predictions = sample_predict(sample_pred_text, pad=False)
print (predictions)

ValueError: Error when checking input: expected embedding_input to have shape (100,) but got array with shape (25,)